In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm 
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

driver.get("https://www.climatempo.com.br/previsao-do-tempo")

time.sleep(5)

In [4]:
links = driver.find_elements(By.XPATH, "//a[contains(@href, '/previsao-do-tempo/agora/cidade/')]")
print(f"Links filtrados: {len(links)}")

Links filtrados: 8439


In [5]:
def extract_code_city_uf(url: str):
    # Pega a parte depois de /cidade/
    parts = url.split("/cidade/")[1].split("/")
    
    code = int(parts[0])   # código numérico da cidade
    city_uf = parts[1]     # ex: "sao-joao-da-boa-vista-sp"
    
    # divide no último hífen: tudo antes é a cidade, os 2 últimos caracteres são a UF
    if "-" in city_uf:
        city, uf = city_uf.rsplit("-", 1)
    else:
        city, uf = city_uf, ""
    
    return code, city, uf


# Testes
assert extract_code_city_uf("https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/3523/vilaboa-go") == (3523, "vilaboa", "go")
assert extract_code_city_uf("https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/6335/vilaflor-rn") == (6335, "vilaflor", "rn")
assert extract_code_city_uf("https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/1234/sao-joao-da-boa-vista-sp") == (1234, "sao-joao-da-boa-vista", "sp")


In [6]:
def get_id_locale(url: str) -> int:
    # Faz a requisição da página
    response = requests.get(url)
    response.raise_for_status()  # erro se não conseguir baixar
    
    # Faz o parse do HTML
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Encontra o div com id="mainContent"
    main_div = soup.find("div", {"id": "mainContent"})
    if not main_div:
        raise ValueError("Div com id='mainContent' não encontrada")
    
    # Extrai o atributo data-id-locale
    data_id = main_div.get("data-id-locale")
    if not data_id:
        raise ValueError("Atributo 'data-id-locale' não encontrado")
    
    return int(data_id)


# Testes
assert get_id_locale("https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/956/xique-xique-ba") == 7995
assert get_id_locale("https://www.climatempo.com.br/previsao-do-tempo/15-dias/cidade/7965/yauco-pr") == 1584

In [7]:
def get_weather(id_locale):
    url = f"https://www.climatempo.com.br/json/myclimatempo/user/weatherNow?idlocale={id_locale}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()


# Teste
weather_json = get_weather(7994)
assert weather_json['data']['getWeatherNow'][0]['data'][0]['locale']['idcity'] == 5354

In [9]:
id_locales = []

for link in tqdm(links, desc="Processando links"):
    pagina_cidade = link.get_attribute("href")
    city_tuple = extract_code_city_uf(pagina_cidade)
    
    if city_tuple and city_tuple[2] == 'pa':  # garante que city_tuple não seja None
        try:
            id_locale = get_id_locale(pagina_cidade)
            id_locales.append(id_locale)
            print(id_locale, city_tuple)
        except ValueError as e:
            print(f"[Aviso] {e} em {pagina_cidade}")
            continue


Processando links:   0%|          | 29/8439 [00:00<01:08, 122.58it/s]

7696 (1191, 'abaetetuba', 'pa')
5916 (6422, 'abelfigueiredo', 'pa')


Processando links:   1%|          | 66/8439 [00:00<00:52, 159.37it/s]

7697 (1192, 'acara', 'pa')
7698 (225, 'afua', 'pa')


Processando links:   2%|▏         | 168/8439 [00:00<00:36, 227.73it/s]

...

7525 (253, 'viseu', 'pa')


Processando links:  99%|█████████▉| 8366/8439 [06:20<00:03, 21.86it/s]

7526 (6479, 'vitoriadoxingu', 'pa')


Processando links: 100%|██████████| 8439/8439 [06:22<00:00, 22.08it/s]

7527 (1221, 'xinguara', 'pa')


In [11]:
temperaturas = []
for id_locale in id_locales:
    weather_json = get_weather(id_locale)
    print(weather_json)

    weather_json = weather_json['data']['getWeatherNow'][0]['data'][0]
    if weather_json['weather']['temperature'] == '-':
        continue
    temperaturas.append({'lat': weather_json['locale']['latitude'],
                         'lon': weather_json['locale']['longitude'],
                          'temperatura': weather_json['weather']['temperature']})
temperaturas

{'data': {'getWeatherNow': [{'data': [{'locale': {'idlocale': 7696, 'idcity': 1191, 'capital': False, 'idcountry': 7, 'ac': 'BR', 'country': 'Brasil', 'uf': 'PA', 'city': 'Abaetetuba', 'region': 'N', 'seaside': False, 'latitude': -1.718, 'longitude': -48.883, 'tourist': False, 'agricultural': False, 'base': 'cities'}, 'weather': {'id': 7696, 'name': 'Abaetetuba', 'date': '04/09/2025', 'dayWeek': 'Quinta-feira', 'dateUpdate': '22:29:00', 'temperature': 29, 'windDirection': 0, 'windVelocity': 8, 'windDirectionDegrees': 237, 'humidity': 82, 'condition': 'Céu limpo', 'pressure': 1012, 'icon': '1n', 'sensation': 34, 'slugCondition': 'céu-limpo'}}]}]}, 'status_code': 200}
{'data': {'getWeatherNow': [{'data': [{'locale': {'idlocale': 5916, 'idcity': 6422, 'capital': False, 'idcountry': 7, 'ac': 'BR', 'country': 'Brasil', 'uf': 'PA', 'city': 'Abel Figueiredo', 'region': 'N', 'seaside': False, 'latitude': -4.954, 'longitude': -48.393, 'tourist': False, 'agricultural': False, 'base': 'cities'}, 

[{'lat': -1.718, 'lon': -48.883, 'temperatura': 29},
 {'lat': -4.954, 'lon': -48.393, 'temperatura': 30},
 {'lat': -1.961, 'lon': -48.197, 'temperatura': 27},
 {'lat': -0.157, 'lon': -50.387, 'temperatura': 27},
 {'lat': -1.942, 'lon': -54.738, 'temperatura': 29},
 {'lat': -0.583, 'lon': -47.567, 'temperatura': 26},
 {'lat': -1.523, 'lon': -52.582, 'temperatura': 28},
 {'lat': -3.203, 'lon': -52.206, 'temperatura': 29},
 {'lat': -0.987, 'lon': -49.94, 'temperatura': 28},
 {'lat': -1.366, 'lon': -48.372, 'temperatura': 28},
 {'lat': -3.472, 'lon': -51.198, 'temperatura': 29},
 {'lat': -1.022, 'lon': -46.635, 'temperatura': 26},
 {'lat': -2.134, 'lon': -47.559, 'temperatura': 26},
 {'lat': -3.606, 'lon': -55.332, 'temperatura': 29},
 {'lat': -1.9, 'lon': -50.164, 'temperatura': 28},
 {'lat': -2.791, 'lon': -49.672, 'temperatura': 29},
 {'lat': -7.348, 'lon': -50.396, 'temperatura': 27},
 {'lat': -1.506, 'lon': -48.626, 'temperatura': 29},
 {'lat': -3.083, 'lon': -51.767, 'temperatura': 2

In [13]:
# Criando DataFrame
df = pd.DataFrame(temperaturas)  # sua lista de dicionários

# Criando geometria de pontos
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Criando mapa centralizado no centro dos pontos
map_center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# Função para definir cor por faixa de temperatura
def cor_por_temperatura(temp):
    if temp < 20:
        return 'blue'
    elif 20 <= temp <= 25:
        return 'yellow'
    else:
        return 'red'

# Adicionando pontos com cor fixa por faixa e tooltip
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=1,
        color=None,
        fill=True,
        fill_color=cor_por_temperatura(row['temperatura']),
        fill_opacity=0.7,
        tooltip=f"Temperatura: {row['temperatura']}°C"
    ).add_to(m)

# HeatMap opcional
heat_data = [[row['lat'], row['lon'], row['temperatura']] for _, row in gdf.iterrows()]
HeatMap(heat_data, radius=25).add_to(m)

# Salvar e exibir via IFrame para contornar o trust
map_file = "../Misc/mapa_temperaturas.html"
m.save(map_file)
IFrame(map_file, width=700, height=500)
